In [ ]:
import os
from PIL import Image
import io
from dotenv import load_dotenv
import logging
import base64


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

load_dotenv()


def process_image(image_path,query):
    try:
        with open(image_path, "rb") as image_file:
            image_data = image_file.read()
            encoded_image = base64.b64encode(image_data).decode('utf-8')

            try:
                img = Image.open(io.BytesIO(image_data))
                img.verify()
            except Exception as e:
                logger.error(f"Invalid image: {str(e)}")
                return f"Invalid image: {str(e)}"

        llama_llb = make_api_request("meta-llama/llama-4-scout-17b-16e-instruct",encoded_image,query)
        llama_90b = make_api_request("meta-llama/llama-4-maverick-17b-128e-instruct",encoded_image,query)

        responses = {}
        for model , response in zip(["llama-4-scout-17b-16e-instruct","llama-4-maverick-17b-128e-instruct"], [llama_llb,llama_90b]):
            if response.status_code == 200:
                result = response.json()
                answer = result["choices"][0]["message"]["content"]
                responses[model] = answer
            else:
                logger.error(f"Error from {model}: {response.status_code} - {response.text}")
                responses[model] = f"Error from {model}: {response.text}"
        
        return responses
            
    except Exception as e:
        logger.error(f"Error processing image: {str(e)}")
        return f"Error processing image: {str(e)}"



def make_api_request(model,base64_image,query):
    messages = [
        {"type": "text", "text": query},
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
            },
        }
    ]
    reponse = requests.post(
        "https://api.groq.com/openai/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {os.getenv('GROQ_API_KEY')}",
            "Content-Type": "application/json",
        },
        json={
            "model": model,
            "messages": messages
        },
        timeout=30
    )
    return reponse
        

    
            
    
